In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile4 = pd.read_pickle("FFF_Train_Decile4.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile4.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile4, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 9s 3ms/step - loss: 0.4430 - acc: 0.8111 - val_loss: 0.3821 - val_acc: 0.8325
Epoch 36/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4224 - acc: 0.8196 - val_loss: 0.3661 - val_acc: 0.8469
Epoch 17/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4263 - acc: 0.8121 - val_loss: 0.3651 - val_acc: 0.8469

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 18/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4260 - acc: 0.8209 - val_loss: 0.3638 - val_acc: 0.8455
Epoch 19/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4101 - acc: 0.8154 - val_loss: 0.3636 - val_acc: 0.8455
Epoch 20/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4107 - acc: 0.8206 - val_loss: 0.3628 - val_acc: 0.8469
Epoch 21/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4143 - acc: 0.8180 - val_loss: 0.3638 - v

3055/3055 [==============================] - 7s 2ms/step - loss: 0.4129 - acc: 0.8196 - val_loss: 0.3629 - val_acc: 0.8455
Epoch 75/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4146 - acc: 0.8232 - val_loss: 0.3628 - val_acc: 0.8469
Test accuracy: 0.846858638431
Train on 3055 samples, validate on 764 samples
Epoch 1/100
3055/3055 [==============================] - 14s 4ms/step - loss: 0.6549 - acc: 0.5964 - val_loss: 0.5426 - val_acc: 0.7304
Epoch 2/100
3055/3055 [==============================] - 8s 3ms/step - loss: 0.5257 - acc: 0.7434 - val_loss: 0.4396 - val_acc: 0.8128
Epoch 3/100
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4827 - acc: 0.7928 - val_loss: 0.4158 - val_acc: 0.8154
Epoch 4/100
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4738 - acc: 0.7921 - val_loss: 0.4114 - val_acc: 0.8259
Epoch 5/100
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4614 - acc: 0.7990 - val_loss: 0.4102 - val_a

3055/3055 [==============================] - 7s 2ms/step - loss: 0.4210 - acc: 0.8206 - val_loss: 0.3659 - val_acc: 0.8455
Epoch 79/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4157 - acc: 0.8196 - val_loss: 0.3656 - val_acc: 0.8455
Epoch 80/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4192 - acc: 0.8141 - val_loss: 0.3654 - val_acc: 0.8455
Epoch 81/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4168 - acc: 0.8170 - val_loss: 0.3657 - val_acc: 0.8455
Epoch 82/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4135 - acc: 0.8167 - val_loss: 0.3654 - val_acc: 0.8455
Epoch 83/100
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4161 - acc: 0.8196 - val_loss: 0.3655 - val_acc: 0.8455
Epoch 84/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4162 - acc: 0.8232 - val_loss: 0.3655 - val_acc: 0.8455
Epoch 85/100
3055/3055 [==============================] - 8s 

Epoch 32/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4068 - acc: 0.8223 - val_loss: 0.3576 - val_acc: 0.8508
Epoch 33/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4030 - acc: 0.8278 - val_loss: 0.3577 - val_acc: 0.8495
Epoch 34/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4026 - acc: 0.8226 - val_loss: 0.3575 - val_acc: 0.8482
Epoch 35/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4044 - acc: 0.8219 - val_loss: 0.3570 - val_acc: 0.8482
Epoch 36/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4066 - acc: 0.8209 - val_loss: 0.3577 - val_acc: 0.8455
Epoch 37/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4062 - acc: 0.8236 - val_loss: 0.3567 - val_acc: 0.8482
Epoch 38/125
3055/3055 [==============================] - 8s 2ms/step - loss: 0.4060 - acc: 0.8252 - val_loss: 0.3571 - val_acc: 0.8482
Epoch 39/125
3055/3055 [========================

3055/3055 [==============================] - 7s 2ms/step - loss: 0.4031 - acc: 0.8288 - val_loss: 0.3557 - val_acc: 0.8469
Epoch 92/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4013 - acc: 0.8232 - val_loss: 0.3556 - val_acc: 0.8455
Epoch 93/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4096 - acc: 0.8196 - val_loss: 0.3557 - val_acc: 0.8469
Epoch 94/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4022 - acc: 0.8265 - val_loss: 0.3558 - val_acc: 0.8495
Epoch 95/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4036 - acc: 0.8226 - val_loss: 0.3559 - val_acc: 0.8482
Epoch 96/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.3991 - acc: 0.8242 - val_loss: 0.3558 - val_acc: 0.8495
Epoch 97/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3996 - acc: 0.8255 - val_loss: 0.3558 - val_acc: 0.8495
Epoch 98/125
3055/3055 [==============================] - 7s 

3055/3055 [==============================] - 8s 3ms/step - loss: 0.4086 - acc: 0.8232 - val_loss: 0.3612 - val_acc: 0.8429
Epoch 22/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4083 - acc: 0.8177 - val_loss: 0.3639 - val_acc: 0.8482

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 23/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4107 - acc: 0.8223 - val_loss: 0.3613 - val_acc: 0.8442
Epoch 24/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4114 - acc: 0.8268 - val_loss: 0.3607 - val_acc: 0.8469
Epoch 25/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4069 - acc: 0.8216 - val_loss: 0.3598 - val_acc: 0.8482
Epoch 26/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4040 - acc: 0.8268 - val_loss: 0.3591 - val_acc: 0.8508
Epoch 27/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4091 - acc: 0.8226 - val_loss: 0.3592 - v

Epoch 46/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4042 - acc: 0.8242 - val_loss: 0.3585 - val_acc: 0.8521
Epoch 47/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4081 - acc: 0.8173 - val_loss: 0.3584 - val_acc: 0.8521
Epoch 48/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4014 - acc: 0.8190 - val_loss: 0.3585 - val_acc: 0.8521
Epoch 49/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4074 - acc: 0.8200 - val_loss: 0.3583 - val_acc: 0.8521
Epoch 50/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4066 - acc: 0.8239 - val_loss: 0.3584 - val_acc: 0.8521
Epoch 51/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4076 - acc: 0.8164 - val_loss: 0.3583 - val_acc: 0.8521
Epoch 52/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4022 - acc: 0.8229 - val_loss: 0.3583 - val_acc: 0.8521
Epoch 53/100
3055/3055 [========================

Epoch 6/50
3055/3055 [==============================] - 3s 976us/step - loss: 0.4549 - acc: 0.8026 - val_loss: 0.3994 - val_acc: 0.8325
Epoch 7/50
3055/3055 [==============================] - 3s 983us/step - loss: 0.4501 - acc: 0.8082 - val_loss: 0.3902 - val_acc: 0.8298
Epoch 8/50
3055/3055 [==============================] - 3s 991us/step - loss: 0.4389 - acc: 0.8088 - val_loss: 0.3858 - val_acc: 0.8338
Epoch 9/50
3055/3055 [==============================] - 3s 983us/step - loss: 0.4413 - acc: 0.8101 - val_loss: 0.3871 - val_acc: 0.8325
Epoch 10/50
3055/3055 [==============================] - 3s 989us/step - loss: 0.4415 - acc: 0.8124 - val_loss: 0.3828 - val_acc: 0.8364
Epoch 11/50
3055/3055 [==============================] - 3s 979us/step - loss: 0.4385 - acc: 0.8033 - val_loss: 0.3764 - val_acc: 0.8351
Epoch 12/50
3055/3055 [==============================] - 3s 971us/step - loss: 0.4437 - acc: 0.8075 - val_loss: 0.3771 - val_acc: 0.8390
Epoch 13/50
3055/3055 [======================

In [3]:
0.93484

0.93484